In [1]:
import numpy as np
import tensorflow as tf

In [2]:
class Weibull:
    def __init__(self, b, k, eps=1e-8):
        self.b = tf.constant(b, dtype=tf.float32)
        self.k = tf.constant(k, dtype=tf.float32)
        self.eps = eps


    def prob(self, x):
        x = tf.clip_by_value(x, self.eps, tf.reduce_max(x))
        return self.b * self.k * tf.math.pow(x, self.k - 1) \
            * tf.math.exp(-self.b * tf.math.pow(x, self.k))


    def log_prob(self, x):
        x = tf.clip_by_value(x, self.eps, tf.reduce_max(x))
        return tf.math.log(self.b) + tf.math.log(self.k) \
            + (self.k - 1) * tf.math.log(x) - self.b * tf.math.pow(x, self.k)
    

    def log_survival(self, x):
        x = tf.clip_by_value(x, self.eps, tf.reduce_max(x))
        return -self.b * tf.math.pow(x, self.k)
    

    def sample(self, size=()):            
        u = tf.random.uniform(minval=0,
                              maxval=1,
                              shape=self.b.shape + size,
                              dtype=tf.float32)
        y = -1 / self.b * tf.math.log(1 - u)
        y = tf.clip_by_value(y, self.eps, tf.reduce_max(y))
        return tf.math.pow(y, 1 / self.k).numpy()


class WeibullMixture:
    def __init__(self, w, b, k, eps=1e-8):
        self.w = w
        self.b = b
        self.k = k
        self.eps = eps


    def prob(self, x):
        x = tf.expand_dims(x, axis=0)
        return tf.reduce_sum(self.w * Weibull(self.b, self.k).prob(x), axis=-1)
    

    def survival(self, x):
        return tf.reduce_sum(self.w * Weibull(self.b, self.k).survival(x))
    

    def log_prob(self, x):
        y = self.prob(x)
        return tf.math.log(tf.clip_by_value(y, self.eps, tf.reduce_max(y)))
    

    def log_survival(self, x):
        y = self.survival(x)
        return tf.math.log(tf.clip_by_value(y, self.eps, tf.reduce_max(y)))

In [3]:
w1 = Weibull(2.0, 3.0)
w2 = Weibull(5.0, 2.0)

weights = tf.constant([0.7, 0.3])
b = tf.constant([2.0, 5.0])
k = tf.constant([3.0, 2.0])
wm = WeibullMixture(weights, b, k)

In [4]:
x = 2.0
(0.7 * w1.prob(x) + 0.3 * w2.prob(x)).numpy()

1.9029578e-06

In [5]:
# il faut que cette ligne renvoie la densité aux différents points
# 2, 3 et 4 pour la compatibilité avec la loi de Weibull classique
wm.prob([2.0, 3.0]).numpy()

array([1.8905909e-06], dtype=float32)